In [1]:
import timm
import torch
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from urllib.request import urlopen
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [2]:
file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePalooza Data/Benchmarking/Beetlepalooza_beetles_image_only.csv"
df = pd.read_csv(file_path, sep="\t")
df = df[['ImageFilePath', 'ScientificName']]
df.head(2)

,ImageFilePath,ScientificName
0,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius aestivus
1,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius aestivus


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = timm.create_model(
    model_name="hf-hub:1aurent/resnet50.tcga_brca_simclr",
    pretrained=True,
).to(device).eval()

data_config = timm.data.resolve_model_data_config(model)

transforms = timm.data.create_transform(**data_config, is_training=False)


In [4]:
def extract_features(image_path) :
    
    image = Image.open(image_path).convert("RGB")
    
    image_tensor = transforms(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        features = model(image_tensor)
    
    return features.cpu().numpy()
    

In [5]:
X = np.vstack([extract_features(img) for img in tqdm(df["ImageFilePath"])])

le = LabelEncoder()

y = le.fit_transform(df["ScientificName"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


100%|██████████| 11399/11399 [01:40<00:00, 113.00it/s]


In [6]:
seed = 99

models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "SVMLinear": SVC(kernel="linear"),
    "SVMPolynomial": SVC(kernel="poly", degree=4),
    "SVMRadialBasis": SVC(kernel="rbf", degree=4),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=seed),    
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), activation='logistic', max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}

for name, model in models.items():
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


NaiveBayes                | Acc: 24.65% | Prec: 42.21% | Rec: 24.65% | F1: 27.90% | Bal Acc: 18.35% | MCC: 0.2280


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


LogisticRegression        | Acc: 80.13% | Prec: 78.30% | Rec: 80.13% | F1: 78.71% | Bal Acc: 49.42% | MCC: 0.7866


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


NearestNeighbor           | Acc: 58.68% | Prec: 57.15% | Rec: 58.68% | F1: 56.22% | Bal Acc: 29.80% | MCC: 0.5549


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


RandomForest              | Acc: 62.50% | Prec: 59.22% | Rec: 62.50% | F1: 57.27% | Bal Acc: 22.87% | MCC: 0.5940
MLP_Baseline              | Acc: 82.63% | Prec: 80.94% | Rec: 82.63% | F1: 81.10% | Bal Acc: 51.60% | MCC: 0.8134


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [7]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_SVMLinear,Pred_SVMPolynomial,Pred_SVMRadialBasis,Pred_NearestNeighbor,Pred_RandomForest,Pred_MLP_Baseline
1766,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Synuchus impunctatus,Cymindis neglecta,Synuchus impunctatus,Calathus advena,Synuchus impunctatus,Calathus advena,Synuchus impunctatus,Calathus advena,Synuchus impunctatus
2629,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Agonum punctiforme,Harpalus amputatus,Synuchus impunctatus,Synuchus impunctatus,Synuchus impunctatus,Calathus advena,Calathus advena,Synuchus impunctatus,Synuchus impunctatus


In [8]:
metrics_df

,Model,Accuracy,Precision,Recall,F1-Score,Balanced Acc,MCC
NaiveBayes,NaiveBayes,0.246491,0.422115,0.246491,0.279046,0.183486,0.228037
LogisticRegression,LogisticRegression,0.801316,0.783018,0.801316,0.787141,0.494199,0.786567
SVMLinear,SVMLinear,0.799561,0.784973,0.799561,0.787507,0.508865,0.784692
SVMPolynomial,SVMPolynomial,0.416228,0.557533,0.416228,0.368688,0.151367,0.378446
SVMRadialBasis,SVMRadialBasis,0.718421,0.670997,0.718421,0.676049,0.283712,0.696315
NearestNeighbor,NearestNeighbor,0.586842,0.571547,0.586842,0.562233,0.298048,0.55487
RandomForest,RandomForest,0.625,0.592151,0.625,0.572703,0.228707,0.593955
MLP_Baseline,MLP_Baseline,0.826316,0.809389,0.826316,0.810958,0.516022,0.813379


In [9]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BeetlePalooza/41.SimCLR-linear-probing-species.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/BeetlePalooza/41.SimCLR-linear-probing-species-metrics.csv", index=False)